# Pump It Up Data Cleaning & Feature Engineering

In [1]:
from matplotlib import pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from sklearn import preprocessing
# from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from category_encoders import BinaryEncoder

In [2]:
# custom helper function to get counts and percents by group
def count_pct(dataframe, column):
    # calculate grouped counts
    grp_count = (
        dataframe.groupby(column)
        .size()
        .reset_index(name = 'count')
        .sort_values(['count'], ascending = False)
        )
    # use counts to generate percents
    grp_pct = grp_count.assign(
        pct = lambda dataframe: dataframe['count'].map(lambda count: count / np.nansum(grp_count['count'])) 
        )
    return grp_pct

def missing_value_plot(data):
    plt.style.use('seaborn')
    plt.figure(figsize = (15,10))
    sns.heatmap(data.isnull(), yticklabels = False, cmap = 'plasma')
    plt.title('Missing Values in Data Set');
    plt.show()

In [224]:
train = pd.read_csv("D:/Projects/pump_it_up/train_values.csv")
train_labels = pd.read_csv("D:/Projects/pump_it_up/train_labels.csv")

In [225]:
# clean up text vars -- all text to lowercase
train = train.applymap(lambda col:col.lower() if type(col) == str else col)

train[['permit']] = pd.to_numeric(train['permit'])

# clean up non-word characters
numeric_cols = train.select_dtypes(include = np.number)

text_cols = train.select_dtypes('object')
text_cols = text_cols.apply(lambda col: col.str.replace('\\W+', '_'), axis = 1)

train = pd.concat([numeric_cols, text_cols], axis = 1, ignore_index = False)

In [226]:
# remove columns with 60% or more missing data -- amount_tsh and num_private
train = train.loc[:, train.isin([' ','NULL',0, np.nan]).mean() < .6]

## Variable Selection

This is based on the exploratory data analysis done in pump_it_up_eda.ipynb

In [176]:
train.columns

Index(['id', 'gps_height', 'longitude', 'latitude', 'region_code',
       'district_code', 'population', 'permit', 'construction_year',
       'date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'recorded_by',
       'scheme_management', 'scheme_name', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

In [227]:
train = train[['funder', 'gps_height',
               'installer', 'longitude', 'latitude', 'date_recorded'
               'basin',  'region_code', 'district_code', 'lga',
               'ward', 'scheme_management', 'management', 
               'permit', 'construction_year',
               'extraction_type_group', 
               'payment',
               'water_quality', 'quantity',
               'source_type','waterpoint_type']]

### Fixing Missing Values

We need to deal with some 0s masquerading as legitimate values before imputation

In [228]:
train.loc[train['construction_year'] == 0, 'construction_year'] = np.nan
train.loc[train['funder'] == '0', 'funder'] = np.nan
train.loc[train['gps_height'] == '0', 'gps_height'] = np.nan
# train.loc[train['district_code'] == '0', 'district_code'] = np.nan

### Categorical Encoding
Chose binary encoding for now-- large number of categories in some vars make one-hot problematic (very high dimensionality, results produce invariate variables, not to mention the practical problem of memory errors on my machine)

In [229]:
be = BinaryEncoder(drop_invariant = True)
be.fit(train)
train = be.transform(train)

### Imputation

In [230]:
imp = IterativeImputer(max_iter=10, max_value = 2013)
imp.fit(train)

train = pd.DataFrame(imp.transform(train), columns = train.columns)
train['construction_year'] = train['construction_year'].round(0)
train['permit'] = train['permit'].round(0)

### Scaling

In [231]:
ss = preprocessing.StandardScaler()
ss.fit(train)

scaled_train = ss.transform(train)


# Modeling

In [121]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as balance_pipe
from sklearn.compose import ColumnTransformer
from sklearn.utils import class_weight

from bayes_opt import BayesianOptimization


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action = 'ignore', category = FutureWarning)

### Set up train and test sets

In [232]:
train_labels = train_labels.assign(
    numeric_status_group = train_labels['status_group'].apply(lambda status_group: (0 if status_group == 'functional' 
                                                                       else 1 if status_group == 'non functional' 
                                                                       else 2)
))
# train.drop('id', axis = 1, inplace = True)
train_labels.drop('id', axis = 1, inplace = True)
train_labels.drop('status_group', axis = 1, inplace = True)


In [233]:
test_set, train_set, test_label, train_label =  train_test_split(scaled_train, 
                                                                 train_labels.values.ravel(), 
                                                                 train_size = .20, 
                                                                 random_state = 123)

In [234]:
test_set.shape

(11880, 98)

In [235]:
dtrain = xgb.DMatrix(train_set, label = train_label)
dtest = xgb.DMatrix(test_set, label = test_label)

In [57]:
count_pct(train_label, 'numeric_status_group')

,numeric_status_group,count,pct
0,0,24187,0.542918
1,1,17162,0.385230
2,2,3201,0.071852


In [109]:
# experimented with smote to balance classes -- no significant improvement
#smote_pipe = balance_pipe([('over_samp', SMOTE(sampling_strategy = {1 : 21000,
#                                                                    2 : 7000}))])

In [115]:
# X, y = smote_pipe.fit_resample(train_set, train_label)
# smote = SMOTE('minority')
# X, y = smote.fit_sample(train_set, train_label)

In [236]:
# adding weights offered a slight boost to performance
cw = class_weight.compute_class_weight('balanced', np.unique(train_label), train_label)

In [124]:
xgb_class = xgb.XGBClassifier(objective = 'multi:softmax', 
                              max_depth = 15, 
                              min_child_weight = 3, 
                              eta = .1, 
                              num_class = 3,
                              num_boost_rounds = 250,
                              early_stopping_rounds = 10,
                              class_weight = cw
                             )
#xgb_class.fit(X, y)
xgb_class.fit(train_set, train_label)

XGBClassifier(base_score=0.5, booster='gbtree',
              class_weight=array([0.61396618, 0.86528377, 4.63917526]),
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=10, eta=0.1, gamma=0, learning_rate=0.1,
              max_delta_step=0, max_depth=15, min_child_weight=3, missing=None,
              n_estimators=100, n_jobs=1, nthread=None, num_boost_rounds=200,
              num_class=3, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [237]:
 params = {
     # Parameters to tune
     'max_depth' : 15,
     'min_child_weight' : 3,
     'eta' : .1,
     'objective' :'multi:softmax',
     'num_class' : 3,
     #'class_weight' : cw,
     'eval_metric' : 'merror' # for multi-class classification problems, use merror or mlogloss
 }

 num_boost_round = 999 # high number of boost rounds allows early_stopping_rounds to do its work

 model = xgb.train(
     params,
     dtrain,
     num_boost_round = num_boost_round,
     evals=[(dtest, "test")],
     early_stopping_rounds = 10
 )

 print("Best MERROR: {:.2f} with {} rounds".format(
                  model.best_score,
                  model.best_iteration+1))

 preds = model.predict(dtest)

[0]	test-merror:0.212121
Will train until test-merror hasn't improved in 10 rounds.
[1]	test-merror:0.208249
[2]	test-merror:0.208333
[3]	test-merror:0.20766
[4]	test-merror:0.206313
[5]	test-merror:0.20404
[6]	test-merror:0.203451
[7]	test-merror:0.202441
[8]	test-merror:0.200084
[9]	test-merror:0.199579
[10]	test-merror:0.200168
[11]	test-merror:0.199411
[12]	test-merror:0.198148
[13]	test-merror:0.197559
[14]	test-merror:0.19798
[15]	test-merror:0.196633
[16]	test-merror:0.19697
[17]	test-merror:0.196717
[18]	test-merror:0.195791
[19]	test-merror:0.194949
[20]	test-merror:0.194192
[21]	test-merror:0.193603
[22]	test-merror:0.193182
[23]	test-merror:0.194024
[24]	test-merror:0.193939
[25]	test-merror:0.193771
[26]	test-merror:0.193603
[27]	test-merror:0.194108
[28]	test-merror:0.193855
[29]	test-merror:0.193266
[30]	test-merror:0.193182
[31]	test-merror:0.193013
[32]	test-merror:0.193182
[33]	test-merror:0.192424
[34]	test-merror:0.192424
[35]	test-merror:0.19234
[36]	test-merror:0.1

In [238]:
preds = model.predict(dtest)

In [239]:
# Attained prediction accuracy on the training set
cm = confusion_matrix(preds, test_label)
acc = cm.diagonal().sum()/cm.sum()
print(acc)

0.807996632996633


In [240]:
cm

array([[5751,  914,  443],
       [ 582, 3551,  146],
       [ 148,   48,  297]], dtype=int64)

### Hyperparameter tuning

In [147]:
dtrain = xgb.DMatrix(scaled_train, label = train_labels)

In [148]:

# Bayesian optimization function for xgboost
# specify the parameters to tune as keyword arguments
def optimize_xgb(max_depth, min_child_weight, gamma, boost_rounds, eta):
    params = {'max_depth': int(max_depth),
              'min_child_weight': int(min_child_weight),
              'subsample': 1,
              'gamma' : gamma,
              'eta': eta,
              'objective' :'multi:softmax',
              'num_class' : 3,
              'eval_metric': 'merror',
              'class_weight' : cw,
              'early_stopping_rounds' : 10
             }
    # Cross validating with the specified parameters in 5 folds and max 250 iterations
    cv_result = xgb.cv(params, dtrain, nfold = 8)
    # return negative merror -- setting to inverse means bayesian maximization returns params that generate lowest merror
    return -1 * cv_result['train-merror-mean'].iloc[-1]


In [149]:
xgb_bo = BayesianOptimization(optimize_xgb, {'max_depth' : (1, 50),
                                             'min_child_weight' : (0, 15),
                                             'gamma' : (0,1)
                                             'boost_rounds' : (1, 500),
                                             'eta' : (.005, .5)
                                            })

In [150]:
xgb_bo.maximize(n_iter = 27, init_points = 8)

|   iter    |  target   | boost_... |   gamma   | max_depth | min_ch... |
-------------------------------------------------------------------------
|  1        | -0.2178   |  248.0    |  0.7077   |  8.94     |  2.599    |
|  2        | -0.09946  |  81.13    |  0.4673   |  19.54    |  3.831    |
|  3        | -0.05393  |  74.6     |  0.3535   |  17.38    |  0.9804   |
|  4        | -0.1174   |  148.4    |  0.8317   |  19.62    |  4.033    |
|  5        | -0.09887  |  69.75    |  0.4496   |  19.66    |  3.496    |
|  6        | -0.09759  |  29.28    |  0.06659  |  18.58    |  3.628    |
|  7        | -0.08399  |  78.86    |  0.9535   |  18.71    |  1.76     |
|  8        | -0.1829   |  42.69    |  0.7063   |  10.88    |  3.489    |
|  9        | -0.2201   |  196.5    |  0.0      |  8.0      |  5.0      |
|  10       | -0.217    |  122.7    |  0.0      |  8.0      |  0.0      |
|  11       | -0.03035  |  220.3    |  0.0      |  20.0     |  0.0      |
|  12       | -0.03035  |  180.3    | 

In [151]:
#Extracting the best parameters
params = xgb_bo.max['params']
print(params)

{'boost_rounds': 220.34581122246493, 'gamma': 0.0, 'max_depth': 20.0, 'min_child_weight': 0.0}


In [ ]:
# third submission

params = {
    # tuned parameters
    'max_depth' : 15,
    'min_child_weight' : 3,
    'gamma' : .0,
    'eta' : .1,
    'subsample' : 1,
    'objective' :'multi:softmax',
    'class_weight' : cw,
    'num_class' : 3,
    'eval_metric' : 'merror' # for multi-class classification problems, use merror or mlogloss
}

#num_boost_round = 250 # high number of boost rounds allows early_stopping_rounds to do its work

model = xgb.train(
    params,
    dtrain,
    num_boost_round = 500,
    early_stopping_rounds = 10,
    evals=[(dtest, "test")]
)

#print("Best MERROR: {:.2f} with {} rounds".format(
#                 model.best_score,
#                 model.best_iteration+1))

#preds = model.predict(dtest)

# Generate Submission

In [241]:

test = pd.read_csv("D:/Projects/pump_it_up/test_values.csv")

test = test[['funder', 'gps_height',
               'installer', 'longitude', 'latitude',
               'basin',  'region_code', 'district_code', 'lga',
               'ward', 'scheme_management', 'management', 
               'permit', 'construction_year',
               'extraction_type_group', 'extraction_type_class',
               'payment',
               'water_quality', 'quality_group', 'quantity', 'quantity_group',
               'source', 'source_type','source_class', 'waterpoint_type', 'waterpoint_type_group']]

 # clean up text vars -- all text to lowercase
test = test.applymap(lambda col:col.lower() if type(col) == str else col)

test[['permit']] = pd.to_numeric(test['permit'])

# clean up non-word characters
numeric_cols = test.select_dtypes(include = np.number)

text_cols = test.select_dtypes('object')
text_cols = text_cols.apply(lambda col: col.str.replace('\\W+', '_'), axis = 1)

test = pd.concat([numeric_cols, text_cols], axis = 1, ignore_index = False)

test.loc[test['construction_year'] == 0, 'construction_year'] = np.nan
test.loc[test['funder'] == '0', 'funder'] = np.nan
test.loc[test['gps_height'] == '0', 'gps_height'] = np.nan
    
# binary encode test set
test = be.transform(test)
    
# impute test set
test = pd.DataFrame(imp.transform(test), columns = test.columns)
test['construction_year'] = test['construction_year'].round(0)
test['permit'] = test['permit'].round(0)
# ensure same column names
test = test.loc[:,train.columns.tolist()]
# scale test set
scaled_test = ss.transform(test)

test_dmatrix = xgb.DMatrix(scaled_test)

submit_preds = model.predict(test_dmatrix)

submit_preds = model.predict(test_dmatrix)

test_ids = pd.read_csv("D:/Projects/pump_it_up/test_values.csv")
test_ids = test_ids[['id']]
test_ids = test_ids.assign(numeric_status_group = submit_preds)

test_ids = test_ids.assign(
    status_group = test_ids['numeric_status_group'].apply(lambda status_group: ('functional' if status_group == 0
                                                                           else 'non functional' if status_group == 1 
                                                                           else 'functional needs repair')
))

pd.DataFrame.to_csv(test_ids[['id', 'status_group']], "D:/Projects/pump_it_up/submit.csv", index = False)